# correction as a function of both reconstructed $E_\nu$ and $Q^2$

In [ ]:
Q2bins = np.linspace(0,0.8,16)
Evbins = np.linspace(0,1.5,16)

In [ ]:
def get_pur_eff_Q2Ev_for_cut(Q2bins=Q2bins,Evbins=Evbins,cut_name = 'PIDa'):

    mup_original = reduced_MCbnbDATAcosmicSamples['no cut']['1mu-1p']
    Q2_pairs_original = mup_original['reco_Q2']
    Ev_pairs_original = mup_original['reco_Ev']    
    h_mup_original,xedges,yedges = np.histogram2d( Q2_pairs_original , Ev_pairs_original , bins=(Q2bins,Evbins) )    
    for x in np.nditer(h_mup_original, op_flags=['readwrite']): x[...] = np.max([1,x])
#     print 'h_mup_original:',h_mup_original
    h = dict()
    for pair_type,label,color in zip(pair_types,MClabels,MCcolors):
        pairs = reduced_MCbnbDATAcosmicSamples[cut_name][pair_type]
#         print "len(pairs):",len(pairs)
        Q2_pairs = pairs['reco_Q2']
        Ev_pairs = pairs['reco_Ev']
        if len(pairs)>2:
            histo,xedges,yedges = np.histogram2d( Q2_pairs , Ev_pairs , bins=(Q2bins,Evbins) )
        else:
            histo = np.zeros((len(Q2bins)-1,len(Evbins)-1))
        for x in np.nditer(histo, op_flags=['readwrite']): x[...] = np.max([1,x])
#         print 'histo:',histo

        h[pair_type] = (np.array(histo)).astype(np.float)
 
    eff_mup = h['1mu-1p']/h_mup_original
    for x in np.nditer(eff_mup, op_flags=['readwrite']): x[...] = np.max([0.01,x])
    
    pur_mup = h['1mu-1p']/(h['1mu-1p'] + h['other pairs'] + h['cosmic'])
    for x in np.nditer(pur_mup, op_flags=['readwrite']): x[...] = np.max([0.001,x])

    
    return eff_mup , pur_mup 

In [ ]:
cuts = ['no cut','vertex activity','soft Pt'] 
cuts_labels = ['no cuts',r'detector cuts',r'kinematical cuts']
eff_mup = dict()
eff_mup_err = dict()
pur_mup = dict()
pur_mup_err = dict()
pur_over_eff_mup = dict()
pur_over_eff_mup_err = dict()

for cut_name in cuts:    
    eff, pur = get_pur_eff_Q2Ev_for_cut(cut_name=cut_name)
    eff_mup[cut_name] = eff
    pur_mup[cut_name] = pur
    pur_over_eff_mup[cut_name] = pur/eff

In [ ]:
h_OnOff_corrected = dict()
h_OnOff_corrected_err = dict()
cuts = ['no cut'] # ,'vertex activity','soft Pt'
for cut_name in cuts:
    
    OnBeamSample = reduced_OnBeam[cut_name]
    OffBeamSample = reduced_OffBeam[cut_name]
    
    h_OnBeam,xedges,yedges = np.histogram2d( OnBeamSample['reco_Q2'] , OnBeamSample['reco_Ev']  , bins=(Q2bins,Evbins) )
    h_OffBeam,xedges,yedges = np.histogram2d( OffBeamSample['reco_Q2'] , OffBeamSample['reco_Ev'] , bins=(Q2bins,Evbins) )
    
    h_OnBeam_minus_OffBeam = h_OnBeam - OffBeam_scaling*h_OffBeam
    h_OnOff_corrected[cut_name] = h_OnBeam_minus_OffBeam*pur_over_eff_mup[cut_name]    

In [ ]:
def CCelasticXsec_mA_fitfunction((Q2,Ev), mA , N):
    N = float(N) # just a normalization constant
    mA = float(mA) # the axial mass
    sigma = N*KinFactor(Ev , Q2) * ( A(Q2,mA) + xi(Ev,Q2)*B(Q2,mA) + np.square(xi(Ev,Q2))*C(Q2,mA) )
    return sigma.ravel()

In [ ]:
hOn,bins=np.histogram(OnBeamSample['reco_Ev'],bins=Evbins)
hOff,bins=np.histogram(OffBeamSample['reco_Ev'],bins=Evbins)
hOnOff = hOn-OffBeam_scaling*hOff
plt.plot(bins[:-1],hOnOff)
print hOnOff

hOn,bins=np.histogram(OnBeamSample['reco_Q2'],bins=Evbins)
hOff,bins=np.histogram(OffBeamSample['reco_Q2'],bins=Evbins)
hOnOff = hOn-OffBeam_scaling*hOff
plt.plot(bins[:-1],hOnOff)
print hOnOff

In [ ]:
bins = (Q2bins,Evbins)
# the On-Off data in a 2d histogram
x_data = OnBeamSample['reco_Q2']
y_data = OnBeamSample['reco_Ev']
OnBeamData = np.array( [x_data , y_data ] )
hOnBeam, edges = np.histogramdd( OnBeamData.T ,bins=bins)
x_data = OffBeamSample['reco_Q2']
y_data = OffBeamSample['reco_Ev']
OffBeamData = np.array( [x_data , y_data ] )
hOffBeam, edges = np.histogramdd( OffBeamData.T ,bins=bins)

hist = hOnBeam - OffBeam_scaling*hOffBeam
# print hist
histCorrected = hist*pur_over_eff_mup[cut_name]

x_centres = (edges[0][:-1] + edges[0][1:])/2
y_centres = (edges[1][:-1] + edges[1][1:])/2
X,Y = np.meshgrid(x_centres,y_centres)
# print (X,Y)


# fit the histogrammed data to the CC-elastic cross-section
histCorrected=histCorrected.ravel()
p0=[1.0,np.mean(histCorrected)]
popt, pcov = opt.curve_fit(CCelasticXsec_mA_fitfunction, (X,Y), histCorrected, p0=p0)
hist_fit = CCelasticXsec_mA_fitfunction( (X , Y), *popt)
print 'initial guess:',p0
print 'popt,pcov:',popt,pcov
# plot....
fig= plt.figure(figsize=(20,6))
ax = fig.add_subplot(1,2,1)
im = ax.imshow(histCorrected.reshape(len(x_centres),len(y_centres))
               ,cmap='hot_r'
               ,origin='bottom'
               ,extent=(x_centres.min(), x_centres.max(), y_centres.min(), y_centres.max())
              )
plt.colorbar(im)
set_axes(ax,x_label=r'reco. $Q^2$ (GeV/c)$^2$'
         ,y_label=r'reco. $E_\nu$ [GeV]'
         ,fontsize=20)


ax = fig.add_subplot(1,2,2)
im = ax.imshow(hist_fit.reshape(len(x_centres),len(y_centres))
               ,cmap='hot_r'
               ,origin='bottom'
               ,extent=(x_centres.min(), x_centres.max(), y_centres.min(), y_centres.max()))
plt.colorbar(im)
ax.contour( X , Y, hist_fit.reshape(len(x_centres),len(y_centres)), 8, colors='w');
set_axes(ax,x_label=r'reco. $Q^2$ (GeV/c)$^2$'
         ,y_label=r'reco. $E_\nu$ [GeV]'
         ,fontsize=20)

In [ ]:
# plot data
# %matplotlib 
X,Y = np.meshgrid(Q2bins[:-1],Evbins[:-1])
fig = plt.figure(figsize=(12,8))
ax = fig.gca(projection='3d')
ax.scatter( X, Y, h_OnOff_corrected[cut_name], c='r', s=50)
set_axes(ax,x_label=r'reco. $Q^2$ (GeV/c)$^2$'
         ,y_label=r'reco. $E_\nu$ [GeV]'
         ,z_label=r'$(On-Off) \times (\epsilon / \mathcal{P}) $'
         ,fontsize=20)

# C,_,_,_ = scipy.linalg.lstsq(CCelasticXsec_mA(kinematics=h_OnOff_corrected[cut_name]), data[:,2])
print X.shape, Y.shape, h_OnOff_corrected[cut_name].shape
ax.plot_surface(X, Y, h_OnOff_corrected[cut_name], rstride=1, cstride=1, alpha=0.2)
# plt.subplot_tool()
# ax.pcolormesh( X,Y,h_OnOff_corrected[cut_name])

In [ ]:
Q2_centers = np.linspace(0.1,0.8,100)
Ev_centers = np.linspace(0.2,1.5,100)
Q2,Ev = np.meshgrid(Q2_centers,Ev_centers)
# imagionary_fit_result = CCelasticXsec_mA_fitfunction( (Q2,E), mA=1. , N=7 )
N = 7
mA = 1.0
sigma = N*KinFactor(Ev , Q2) * ( A(Q2,mA) + xi(Ev,Q2)*B(Q2,mA) + np.square(xi(Ev,Q2))*C(Q2,mA) )
# imagionary_fit_result = imagionary_fit_result.ravel()
# # plot....
# fig= plt.figure(figsize=(12,6))
# ax = fig.add_subplot(1,1,1)
# im = ax.imshow(imagionary_fit_result.reshape(len(x_centers),len(y_centers))
#                ,cmap='hot_r'
#                ,origin='bottom'
#                ,extent=(x_centers.min(), x_centers.max(), y_centers.min(), y_centers.max()))
# plt.colorbar(im)
# ax.contour( X , Y, imagionary_fit_result.reshape(len(x_centers),len(y_centers)), 8, colors='w');
# set_axes(ax,x_label=r'reco. $Q^2$ (GeV/c)$^2$'
#          ,y_label=r'reco. $E_\nu$ [GeV]'
#          ,fontsize=20)
print sigma.shape
hQ2 = sigma.sum(axis=0)
hEv = sigma.sum(axis=1)
plt.plot(Q2_centers,hQ2)
plt.plot(Ev_centers,hEv)